In [25]:
#!/usr/bin/env python3
import pulp

# Bipolar values: mapping logic 0 -> -1 and logic 1 -> +1
bipolar_values = [-1, 1]

# Define a truth table for a 2-input logic function.
# For example, for an OR gate:
truth_table = {
    (-1, -1): -1, 
    (-1,  1):  1, 
    ( 1, -1):  1, 
    ( 1,  1): -1  
}

def is_valid_state(a, b, y, truth_table):
    """
    Check if the state (a, b, y) is valid according to the provided truth table.
    Inputs a and b are the inputs, and y is the output (all in bipolar form).
    """
    # Look up the expected output from the truth table using (a, b) as the key.
    expected_y = truth_table[(a, b)]
    return y == expected_y

def main():
    # Create the LP problem to maximize the energy gap d.
    prob = pulp.LpProblem("Hamiltonian_Invertible_Logic", pulp.LpMaximize)
    
    # Define LP variables for biases (h) for nodes A, B, and Y.
    h0 = pulp.LpVariable("h0", lowBound=None, cat='Continuous')
    h1 = pulp.LpVariable("h1", lowBound=None, cat='Continuous')
    h2 = pulp.LpVariable("h2", lowBound=None, cat='Continuous')
    
    # Define LP variables for weights (J) between nodes.
    J01 = pulp.LpVariable("J01", lowBound=None, cat='Continuous')
    J02 = pulp.LpVariable("J02", lowBound=None, cat='Continuous')
    J12 = pulp.LpVariable("J12", lowBound=None, cat='Continuous')
    
    # E_min: energy for valid states; d: energy gap between valid and invalid states.
    E_min = pulp.LpVariable("E_min", lowBound=None, cat='Continuous')
    d = pulp.LpVariable("d", lowBound=0, cat='Continuous')
    
    # Objective: maximize the energy gap d.
    prob += d, "Maximize_energy_gap_d"
    
    # Normalization: Fix E_min to a constant (e.g., -3) to remove scaling freedom.
    prob += E_min == -3, "Fix_E_min_to_-3"
    
    # Iterate over all 2^3 = 8 possible states (for A, B, and Y).
    for a in bipolar_values:
        for b in bipolar_values:
            for y in bipolar_values:
                # The energy expression:
                # H = - (h0*a + h1*b + h2*y) - (J01*a*b + J02*a*y + J12*b*y)
                energy_expr = - (h0 * a + h1 * b + h2 * y) - (J01 * a * b + J02 * a * y + J12 * b * y)
                
                # Use the truth table to decide if the state is valid.
                if is_valid_state(a, b, y, truth_table):
                    # Valid states must have energy equal to E_min.
                    prob += (energy_expr == E_min), f"Valid_state_a{a}_b{b}_y{y}"
                else:
                    # Invalid states must have energy at least E_min + d.
                    prob += (energy_expr >= E_min + d), f"Invalid_state_a{a}_b{b}_y{y}"
    
    # Solve the LP.
    prob.solve()
    
    # Display results.
    print("Status:", pulp.LpStatus[prob.status])
    print("Optimal energy gap d =", pulp.value(d))
    print("E_min =", pulp.value(E_min))
    print("Bias values:")
    print("  h0 =", pulp.value(h0))
    print("  h1 =", pulp.value(h1))
    print("  h2 =", pulp.value(h2))
    print("Weight values:")
    print("  J01 =", pulp.value(J01))
    print("  J02 =", pulp.value(J02))
    print("  J12 =", pulp.value(J12))

if __name__ == '__main__':
    main()


Status: Infeasible
Optimal energy gap d = 3.0
E_min = -3.0
Bias values:
  h0 = 0.0
  h1 = 0.0
  h2 = 0.0
Weight values:
  J01 = 0.0
  J02 = 0.0
  J12 = 0.0


In [27]:
#!/usr/bin/env python3
import pulp

# Bipolar values: logic 0 -> -1, logic 1 -> +1
bipolar_values = [-1, 1]

# Define the truth table for XOR (for a 2-input gate) in bipolar form.
# For inputs (A, B), the expected output Y is:
# (-1, -1): -1, (-1, +1): +1, (+1, -1): +1, (+1, +1): -1.
truth_table = {
    (-1, -1): -1,  # 0 XOR 0 = 0
    (-1,  1):  1,  # 0 XOR 1 = 1
    ( 1, -1):  1,  # 1 XOR 0 = 1
    ( 1,  1): -1   # 1 XOR 1 = 0
}

def is_valid_state_xor(a, b, y, truth_table):
    """
    Checks if (a, b, y) is valid according to the XOR truth table.
    """
    expected_y = truth_table[(a, b)]
    return y == expected_y

def main():
    # Create the LP problem to maximize the energy gap d.
    prob = pulp.LpProblem("Hamiltonian_Invertible_XOR", pulp.LpMaximize)
    
    # Define LP variables for biases for four nodes:
    # Node 0: Input A, Node 1: Input B, Node 2: Auxiliary, Node 3: Output Y.
    h0 = pulp.LpVariable("h0", lowBound=None, cat='Continuous')
    h1 = pulp.LpVariable("h1", lowBound=None, cat='Continuous')
    h2 = pulp.LpVariable("h2", lowBound=None, cat='Continuous')
    h3 = pulp.LpVariable("h3", lowBound=None, cat='Continuous')
    
    # Define LP variables for weights between nodes (only one for each pair, i<j).
    J01 = pulp.LpVariable("J01", lowBound=None, cat='Continuous')
    J02 = pulp.LpVariable("J02", lowBound=None, cat='Continuous')
    J03 = pulp.LpVariable("J03", lowBound=None, cat='Continuous')
    J12 = pulp.LpVariable("J12", lowBound=None, cat='Continuous')
    J13 = pulp.LpVariable("J13", lowBound=None, cat='Continuous')
    J23 = pulp.LpVariable("J23", lowBound=None, cat='Continuous')
    
    # Define E_min (energy for valid states) and d (energy gap).
    E_min = pulp.LpVariable("E_min", lowBound=None, cat='Continuous')
    d = pulp.LpVariable("d", lowBound=0, cat='Continuous')
    
    # Objective: maximize the energy gap d.
    prob += d, "Maximize_energy_gap_d"
    
    # Normalization: fix E_min to -3.
    prob += E_min == -3, "Fix_E_min_to_-3"
    
    # Enumerate over all 16 states for nodes A, B, auxiliary, and Y.
    for a in bipolar_values:
        for b in bipolar_values:
            for aux in bipolar_values:
                for y in bipolar_values:
                    # Energy expression:
                    # H = - (h0*a + h1*b + h2*aux + h3*y)
                    #   - (J01*a*b + J02*a*aux + J03*a*y + J12*b*aux + J13*b*y + J23*aux*y)
                    energy_expr = - (h0 * a + h1 * b + h2 * aux + h3 * y) \
                                  - (J01 * a * b + J02 * a * aux + J03 * a * y \
                                  + J12 * b * aux + J13 * b * y + J23 * aux * y)
                    
                    # For valid (A, B, Y) combinations per the truth table...
                    if is_valid_state_xor(a, b, y, truth_table):
                        # ...if we choose aux = +1 as the canonical valid auxiliary value,
                        # enforce that energy equals E_min.
                        if aux == 1:
                            prob += (energy_expr == E_min), f"Valid_state_a{a}_b{b}_aux{aux}_y{y}"
                        else:
                            # For aux = -1, allow a higher energy.
                            prob += (energy_expr >= E_min + d), f"Invalid_aux_a{a}_b{b}_aux{aux}_y{y}"
                    else:
                        # For invalid (A, B, Y) combinations, require energy at least E_min + d.
                        prob += (energy_expr >= E_min + d), f"Invalid_state_a{a}_b{b}_aux{aux}_y{y}"
    
    # Solve the LP.
    prob.solve()
    
    # Display results.
    print("Status:", pulp.LpStatus[prob.status])
    print("Optimal energy gap d =", pulp.value(d))
    print("E_min =", pulp.value(E_min))
    print("Bias values:")
    print("  h0 =", pulp.value(h0))
    print("  h1 =", pulp.value(h1))
    print("  h2 =", pulp.value(h2))
    print("  h3 =", pulp.value(h3))
    print("Weight values:")
    print("  J01 =", pulp.value(J01))
    print("  J02 =", pulp.value(J02))
    print("  J03 =", pulp.value(J03))
    print("  J12 =", pulp.value(J12))
    print("  J13 =", pulp.value(J13))
    print("  J23 =", pulp.value(J23))

if __name__ == '__main__':
    main()


Status: Optimal
Optimal energy gap d = 6.0000893e-12
E_min = -3.0
Bias values:
  h0 = 0.0
  h1 = 0.0
  h2 = 3.0
  h3 = 0.0
Weight values:
  J01 = 0.0
  J02 = -2.0001778e-12
  J03 = -9.9986686e-13
  J12 = -1.0000889e-12
  J13 = -1.9999558e-12
  J23 = 0.0


In [28]:
#!/usr/bin/env python3
import pulp

# Bipolar values: mapping logic 0 -> -1 and logic 1 -> +1
bipolar_values = [-1, 1]

# Define the truth table for an inverter (NOT gate) in bipolar form.
# For an inverter:
#   NOT(-1) = 1
#   NOT(1)  = -1
truth_table = {
    -1: 1,
     1: -1
}

def is_valid_inverter(a, y, truth_table):
    """
    Check if the state (a, y) is valid according to the inverter truth table.
    'a' is the input and 'y' is the output (both in bipolar form).
    """
    expected_y = truth_table[a]
    return y == expected_y

def main():
    # Create the LP problem to maximize the energy gap d.
    prob = pulp.LpProblem("Hamiltonian_Inverter", pulp.LpMaximize)
    
    # Define LP variables for biases for the input (A) and output (Y) nodes.
    h0 = pulp.LpVariable("h0", lowBound=None, cat='Continuous')  # Bias for input A
    h1 = pulp.LpVariable("h1", lowBound=None, cat='Continuous')  # Bias for output Y
    
    # Define LP variable for the weight (J) between nodes A and Y.
    J01 = pulp.LpVariable("J01", lowBound=None, cat='Continuous')
    
    # E_min: energy for valid states; d: energy gap between valid and invalid states.
    E_min = pulp.LpVariable("E_min", lowBound=None, cat='Continuous')
    d = pulp.LpVariable("d", lowBound=0, cat='Continuous')
    
    # Objective: maximize the energy gap d.
    prob += d, "Maximize_energy_gap_d"
    
    # Normalization: fix E_min to a constant (e.g., -1) to remove scaling freedom.
    prob += E_min == -1, "Fix_E_min_to_-1"
    
    # Iterate over all 2^2 = 4 possible states for input A and output Y.
    for a in bipolar_values:
        for y in bipolar_values:
            # The energy expression:
            # H = - (h0*a + h1*y) - (J01*a*y)
            energy_expr = - (h0 * a + h1 * y) - (J01 * a * y)
            
            # Use the truth table to decide if the state (a, y) is valid.
            if is_valid_inverter(a, y, truth_table):
                # Valid states must have energy equal to E_min.
                prob += (energy_expr == E_min), f"Valid_state_a{a}_y{y}"
            else:
                # Invalid states must have energy at least E_min + d.
                prob += (energy_expr >= E_min + d), f"Invalid_state_a{a}_y{y}"
    
    # Solve the LP.
    prob.solve()
    
    # Display results.
    print("Status:", pulp.LpStatus[prob.status])
    print("Optimal energy gap d =", pulp.value(d))
    print("E_min =", pulp.value(E_min))
    print("Bias values:")
    print("  h0 =", pulp.value(h0))
    print("  h1 =", pulp.value(h1))
    print("Weight value:")
    print("  J01 =", pulp.value(J01))

if __name__ == '__main__':
    main()


Status: Optimal
Optimal energy gap d = 2.0
E_min = -1.0
Bias values:
  h0 = -0.0
  h1 = -0.0
Weight value:
  J01 = -1.0


In [2]:
import pulp

def compute_hamiltonian(truth_table):
    """
    Computes the Hamiltonian parameters (h, J) for an arbitrary logic function.

    Args:
    - truth_table: A list of tuples [(inputs, output, is_valid), ...]
        inputs -> List of -1 or +1 (bipolar representation of input bits)
        output -> -1 or +1 (bipolar representation of output bit)
        is_valid -> Boolean indicating whether this state is valid

    Returns:
    - Optimized values for h and J
    """
    num_bits = len(truth_table[0][0]) + 1  # Number of bits including output

    # Define LP problem
    problem = pulp.LpProblem("General_Logic_Hamiltonian", pulp.LpMaximize)

    # Define LP variables for h and J
    h = pulp.LpVariable.dicts('h', range(num_bits), -5, 5, 'Continuous')
    J = pulp.LpVariable.dicts('J', [(i, j) for i in range(num_bits) for j in range(i+1, num_bits)], -5, 5, 'Continuous')

    # Define energy separation variable
    d = pulp.LpVariable('d', 0.01, None, 'Continuous')  # Ensure meaningful separation

    # Define minimum energy variable
    E_min = pulp.LpVariable('E_min', None, None, 'Continuous')

    # Define constraints based on truth table
    for state, output, is_valid in truth_table:
        full_state = state + [output]  # Combine input and output bits

        # Compute Hamiltonian energy for this state
        H_h = pulp.lpSum(h[i] * full_state[i] for i in range(num_bits))
        H_j = pulp.lpSum(J[(i, j)] * full_state[i] * full_state[j] for i in range(num_bits) for j in range(i+1, num_bits))
        energy = H_h + H_j

        if is_valid:
            problem += energy == E_min  # Valid states get minimum energy
        else:
            problem += energy >= E_min + d  # Invalid states have higher energy

    # Objective function: Maximize d to improve separation
    problem += d

    # Solve the optimization problem
    status = problem.solve(pulp.PULP_CBC_CMD(msg=1))

    # Check solver status
    if status != 1:
        print("Solver did not converge.")
        return None

    # Extract optimized h and J values
    h_values = {i: h[i].varValue for i in range(num_bits)}
    J_values = {(i, j): J[(i, j)].varValue for i in range(num_bits) for j in range(i+1, num_bits)}

    return h_values, J_values
# Truth table for XOR gate in bipolar representation
xor_truth_table = [
    ([-1, -1], -1, True),   # 0 XOR 0 = 0 (Valid)
    ([-1,  1],  1, True),   # 0 XOR 1 = 1 (Valid)
    ([ 1, -1],  1, True),   # 1 XOR 0 = 1 (Valid)
    ([ 1,  1], -1, True),   # 1 XOR 1 = 0 (Valid)
    ([-1, -1],  1, False),  # 0 XOR 0 ≠ 1 (Invalid)
    ([-1,  1], -1, False),  # 0 XOR 1 ≠ 0 (Invalid)
    ([ 1, -1], -1, False),  # 1 XOR 0 ≠ 0 (Invalid)
    ([ 1,  1],  1, False)   # 1 XOR 1 ≠ 1 (Invalid)
]

# Compute Hamiltonian parameters
h, J = compute_hamiltonian(xor_truth_table)

# Display results
print("Computed Bias Terms (h):", h)
print("Computed Interaction Terms (J):", J)


Solver did not converge.


TypeError: cannot unpack non-iterable NoneType object